In [ ]:
path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M5082_clear.gpx"

In [6]:
import xml.etree.ElementTree as ET
import folium

import os

folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes"

# List all files in the folder
files = os.listdir(folder_path)

# Define the bridge ID
bridge_id = 'M1075'

# Filter the files for the specific bridge
clear_route = None
alternative_route = None

for file in files:
    if file.startswith(bridge_id):
        if "_clear" in file:
            clear_route = os.path.join(folder_path, file)
        elif "_alt" in file:
            alternative_route = os.path.join(folder_path, file)

# Assign variables for clear and alternative routes
print(f"Clear route for {bridge_id}: {clear_route}")
print(f"Alternative route for {bridge_id}: {alternative_route}")

Clear route for M1075: C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M1075_clear.gpx
Alternative route for M1075: C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M1075_alt.gpx


In [10]:
import plotly.graph_objs as go

# Extract elevation profile data from the GPX file


def extract_elevation_data(gpx_file):
    elevations = []
    distances = []
    with open(gpx_file, "r", encoding="utf-8") as file:
        content = file.read()
        # Split into trkpt segments
        trkpt_segments = content.split("<trkpt")
        previous_lat, previous_lon = None, None
        total_distance = 0.0
        for segment in trkpt_segments[1:]:
            # Extract latitude and longitude
            lat_start = segment.find('lat="') + 5
            lat_end = segment.find('"', lat_start)
            lon_start = segment.find('lon="') + 5
            lon_end = segment.find('"', lon_start)
            ele_start = segment.find("<ele>") + 5
            ele_end = segment.find("</ele>")

            if lat_start > 4 and lon_start > 4 and ele_start > 4:
                latitude = float(segment[lat_start:lat_end])
                longitude = float(segment[lon_start:lon_end])
                elevation = float(segment[ele_start:ele_end])

                if previous_lat is not None and previous_lon is not None:
                    # Calculate distance between points (simple approximation)
                    from math import radians, sin, cos, sqrt, atan2
                    R = 6371000  # Earth radius in meters
                    lat1, lon1 = radians(previous_lat), radians(previous_lon)
                    lat2, lon2 = radians(latitude), radians(longitude)
                    dlat = lat2 - lat1
                    dlon = lon2 - lon1
                    a = sin(dlat / 2)**2 + cos(lat1) * \
                        cos(lat2) * sin(dlon / 2)**2
                    c = 2 * atan2(sqrt(a), sqrt(1 - a))
                    distance = R * c
                    total_distance += distance

                elevations.append(elevation)
                # Convert to kilometers
                distances.append(total_distance / 1000)

                previous_lat, previous_lon = latitude, longitude

    return distances, elevations

In [25]:
# Path to the clear and alternative GPX files
clear_gpx_file = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M1075_clear.gpx"
alt_gpx_file = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M1075_alt.gpx"

# Extract data for both routes
clear_distances, clear_elevations = extract_elevation_data(clear_gpx_file)
alt_distances, alt_elevations = extract_elevation_data(alt_gpx_file)

# Create a Plotly figure
fig = go.Figure()

# Add the alternative route's elevation profile
fig.add_trace(go.Scatter(
    x=alt_distances,
    y=alt_elevations,
    mode="lines",
    name="Alternative Route",
    line=dict(color="red"),
    fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
    fillcolor='rgba(255, 99, 71, 0.3)'  # Transparent red fill
))

# Add the clear route's elevation profile
fig.add_trace(go.Scatter(
    x=clear_distances,
    y=clear_elevations,
    mode="lines",
    name="Clear Route",
    line=dict(color="blue"),
    fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
    fillcolor='rgba(135, 206, 250, 0.3)'  # Transparent blue fill
))



# Update the layout
fig.update_layout(
    title="Výškové profily priamej trasy a obchádzky",
    xaxis_title="Vzdialenosť (km)",
    yaxis_title="Nadmorská výška (m)",
    legend=dict(
        title="Typ",
        orientation="h",  # Horizontal legend
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.25
    ),
    template="plotly_white"
)

# Show the plot
fig.show()